In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import polars as pl
from sqlalchemy import create_engine,text
from scipy import stats
import plotly.express as px
import toml
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import h5py

%matplotlib inline
from IPython.display import display, HTML

relative_path = "../../../.."

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:0,.0f}'.format

## Transit Boardings

In [2]:

df = pd.read_csv(os.path.join(relative_path,'outputs','transit','daily_boardings_by_agency.csv'),index_col=0)
df.loc['Region Total','boardings'] = df['boardings'].sum()
df = df.reset_index()
df.rename(columns={'agency_name': 'Agency', 'boardings': 'Boardings'}, inplace=True)
HTML(df.to_html(index=False))

Agency,Boardings
King County Metro,"752,535"
Sound Transit,"538,696"
Community Transit,"99,580"
Pierce Transit,"67,950"
Kitsap Transit,"42,667"
Washington Ferries,"24,882"
Everett Transit,"16,326"
Region Total,"1,542,635"


In [3]:
df = pd.read_csv(os.path.join(relative_path,'outputs','transit','transit_line_results.csv'),index_col=0)

transit_type_map = {
    1: 'Local Bus',
    2: 'Express Bus',
    3: 'Bus Rapid Transit',
    4: 'Streetcar',
    5: 'Commuter Rail',
    6: 'Light Rail',
    7: 'Auto Ferry',
    8: 'Passenger Ferry'
}
df['transit_type'] = df['transit_type'].map(transit_type_map)
df = df.groupby('transit_type').sum()[['boardings']]
df.loc['Total'] = df.sum(axis=0)
df = df.reset_index()
df.rename(columns={'boardings':' Boardings','transit_type':'Transit Type'}, inplace=True)
HTML(df.to_html(index=False))

Transit Type,Boardings
Auto Ferry,"39,042"
Bus Rapid Transit,"326,236"
Commuter Rail,"10,136"
Light Rail,"492,102"
Local Bus,"653,710"
Passenger Ferry,"10,851"
Streetcar,"10,557"
Total,"1,542,635"


In [4]:
df = pd.read_csv(os.path.join(relative_path,'outputs','transit','transit_line_results.csv'),index_col=0)
df = df[(df['agency_code']==6)].groupby('transit_type').sum()[['boardings']]
df.index = df.index.map({1: 'Bus', 3: 'BRT (Stride)', 5: 'Commuter Rail', 6: 'Light Rail'})
df.rename(columns={'boardings':' Boardings'}, inplace=True)

df

,Boardings
transit_type,
Bus,"7,479"
BRT (Stride),"28,978"
Commuter Rail,"10,136"
Light Rail,"492,102"


In [5]:
# all transit stop locations
df_transit_stops = pd.read_csv(os.path.join(relative_path,'inputs','scenario','networks','transit_stops.csv'), usecols = ['PSRCJunctID', 'x', 'y','light_rail'])

# Load station names
conn = create_engine('sqlite:///../../../../inputs/db/'+config['db_name'])
df_lr_station_names = pd.read_sql( "SELECT * FROM light_rail_stations_2050", con=conn)

# Merge station names to df 
light_rail_stations = df_transit_stops[df_transit_stops['light_rail']==1].\
    merge(df_lr_station_names, left_on=['PSRCJunctID'], right_on=['Station Node'])

In [6]:
# Light rail boardings by station
df_boardings = pd.read_csv(os.path.join(relative_path,'outputs','transit','boardings_by_stop.csv'))
df = pd.merge(df_boardings, light_rail_stations, left_on=['i_node'], right_on=['PSRCJunctID'])

df = df[['total_boardings','Station Name','PSRCJunctID']]#.drop_duplicates()

# pivot table of boardings by station 
df[['PSRCJunctID','Station Name','total_boardings']]

,PSRCJunctID,Station Name,total_boardings
0,197649,Husky Stadium,"10,634"
1,198007,Westlake,"35,109"
2,198009,Symphony,"19,031"
3,198011,Pioneer Square,"16,772"
4,198013,Int'l District,"28,289"
5,198014,Capitol Hill,"16,385"
6,198015,Stadium,"2,443"
7,198016,SODO,"12,613"
8,198017,Beacon Hill,"3,378"
9,198018,Mount Baker,"3,453"


In [7]:
# Light rail boardings by segment
df = pd.read_csv(os.path.join(relative_path,'outputs/transit/transit_line_results.csv'), usecols = ['line_id','mode','description'])
df_light_rail = df[(df['mode']=="r") & (df['description']!="Streetcar-South Lake")].copy().\
    drop_duplicates()
df_light_rail['description'] = df_light_rail['description'].str[:6]
df_light_rail.rename(columns={'description':'Line'}, inplace=True)

In [8]:
# Light rail boardings by segment
df_boardings = pd.read_csv(os.path.join(relative_path,'outputs','transit','transit_segment_results.csv'))

# get line and depart station names
df = df_boardings.merge(df_lr_station_names, left_on=['i_node'], right_on=['Station Node']).\
    merge(df_light_rail, on=['line_id'])

# all station boardings by line
board_station_line = df.groupby(['Line','Station Name'])['segment_boarding'].sum().reset_index()

In [9]:
board_station_line.groupby('Line')['segment_boarding'].sum().\
    reset_index().set_index('Line').\
    rename(columns={'segment_boarding':'Total Boardings'})

,Total Boardings
Line,
1 Line,"131,748"
2 Line,"178,533"
3 Line,"143,665"
4 Line,"21,309"
T Line,"16,848"


In [10]:
def get_stop_boardings(line_name):
    df = board_station_line[board_station_line['Line']==line_name]
    df2 = df.groupby('Line').sum()['segment_boarding'].reset_index()
    df2['Station Name'] = 'Total'
    return pd.concat([df2[["Line", "Station Name", "segment_boarding"]],df], ignore_index=True)

get_stop_boardings('1 Line')

,Line,Station Name,segment_boarding
0,1 Line,Total,"131,748"
1,1 Line,Angle Lake,"3,956"
2,1 Line,Ballard Station,"5,790"
3,1 Line,Beacon Hill,"3,378"
4,1 Line,Boeing Access Station,3
5,1 Line,Columbia City,"2,285"
6,1 Line,Denny Station,"5,712"
7,1 Line,East Tacoma Station,"2,511"
8,1 Line,Federal Way Station,"8,172"
9,1 Line,Fife Station,"2,671"


In [11]:
get_stop_boardings('2 Line')

,Line,Station Name,segment_boarding
0,2 Line,Total,"178,533"
1,2 Line,Alderwood Station,"3,021"
2,2 Line,Ash Way Station,"2,930"
3,2 Line,BelRed Station,"5,254"
4,2 Line,Bellevue Downtown Station,"12,815"
5,2 Line,Capitol Hill,"8,347"
6,2 Line,Downtown Redmond Station,"5,577"
7,2 Line,East Main Station,"9,137"
8,2 Line,Husky Stadium,"5,029"
9,2 Line,Int'l District,"10,630"


In [12]:
get_stop_boardings('3 Line')

,Line,Station Name,segment_boarding
0,3 Line,Total,"143,665"
1,3 Line,Alderwood Station,"3,360"
2,3 Line,Ash Way Station,"3,647"
3,3 Line,Avalon Station,"2,647"
4,3 Line,Capitol Hill,"8,037"
5,3 Line,Delridge Station,"6,552"
6,3 Line,Everett Station,"6,048"
7,3 Line,Husky Stadium,"5,605"
8,3 Line,Int'l District,"6,622"
9,3 Line,Lynnwood Station,"5,817"


In [13]:
get_stop_boardings('4 Line')

,Line,Station Name,segment_boarding
0,4 Line,Total,"21,309"
1,4 Line,Bellevue Downtown Station,"4,597"
2,4 Line,East Main Station,"5,232"
3,4 Line,Eastgate Station,"3,109"
4,4 Line,Issaquah Station,"3,032"
5,4 Line,Richards Rd Station,"2,768"
6,4 Line,South Kirkland Station,"1,193"
7,4 Line,Wilburton Station,"1,378"


In [14]:
get_stop_boardings('T Line')

,Line,Station Name,segment_boarding
0,T Line,Total,"16,848"
1,T Line,6th Ave Station,"1,600"
2,T Line,Ainsworth Station,642
3,T Line,Commerce Station,515
4,T Line,Convention Center Station,309
5,T Line,Hilltop District Station,650
6,T Line,Old City Hall Station,374
7,T Line,Pearl Station,"1,097"
8,T Line,S 25th St Station,"1,235"
9,T Line,S 4th St Station,432


In [15]:
df = pd.read_csv(os.path.join(relative_path,'outputs','transit','total_transit_trips.csv'),index_col=0)
df.rename(index={'commuter_rail': 'Commuter Rail',
                'litrat': 'Light Rail',
                'ferry': 'Auto Ferry',
                'passenger_ferry': 'Passenger Ferry',
                'trnst': 'Bus'}, inplace=True)
df.columns = ['Total Trips']
df.loc['Total'] = df.sum(axis=0)
df.reset_index(inplace=True)
df.rename(columns={'index': 'Transit Type'}, inplace=True)
HTML(df.to_html(index=False))

Transit Type,Total Trips
Commuter Rail,"11,188"
Light Rail,"454,985"
Auto Ferry,"42,146"
Passenger Ferry,"5,362"
Bus,"546,720"
Total,"1,060,402"


## Households Near HCT

In [16]:
# List of Stations
# Load transit stops file

#"The definition we have been using for the RTP is BRT, LRT, Commuter Rail, StreetCar and Ferry."
df = pd.read_csv(r'../../../../inputs/scenario/networks/transit_stops.csv')
# Streetcar is coded as light rail
df['hct'] = 0
df.loc[df[['commuter_rail','light_rail','ferry','brt']].sum(axis=1) > 0,'hct'] = 1
df_hct = df[df['hct'] == 1]

# Map of Stations
# Load as a geodataframe
gdf_hct = gpd.GeoDataFrame(
    df_hct, geometry=gpd.points_from_xy(df_hct.x, df_hct.y))

gdf_hct.crs = 'EPSG:2285'

In [17]:
df_lu = pl.read_csv(r'..\..\..\..\inputs\scenario\landuse\parcels_urbansim.txt', separator=" ").to_pandas()

# Load as a geodataframe
gdf_lu = gpd.GeoDataFrame(
    df_lu, geometry=gpd.points_from_xy(df_lu.xcoord_p, df_lu.ycoord_p))

gdf_lu.crs = 'EPSG:2285'

In [18]:
parcel_geog = pd.read_sql_table('parcel_'+config['base_year']+'_geography', 'sqlite:///../../../../inputs/db/'+config['db_name'])

In [19]:
df_lu = df_lu.merge(parcel_geog, left_on='parcelid', right_on='ParcelID', how='left')
# Add a field that defines whether a parcel is inside (1) or outside (0) any RGC
df_lu['RGC_binary'] = 0
df_lu.loc[df_lu['GrowthCenterName'] != 'Not in RGC', 'RGC_binary'] = 1

In [20]:
def calculate_buffer(gdf_lu, _gdf_hct, distance):
    
    # Buffer the HCT station gdf
    _gdf_hct['geometry'] = _gdf_hct.buffer(distance)

    gdf_intersect = gpd.overlay(_gdf_hct, gdf_lu, how="intersection", keep_geom_type=False)
    df = df_lu[df_lu['parcelid'].isin(gdf_intersect['parcelid'].unique())]
    
    return df, gdf_intersect

def aggregate_parcels(df, col_dict, sum_field):
    results_df = pd.DataFrame()
    for col, name in col_dict.items():
        _df = df[[col,sum_field]].groupby(col).sum()[[sum_field]]
        _df['group'] = name
        results_df = pd.concat([results_df,_df])
    results_df = results_df.reset_index()
    
    return results_df

In [21]:
df_025, gdf_025 = calculate_buffer(gdf_lu, gdf_hct.copy(), distance=5280.0/4)

In [22]:
df_050, gdf_050 = calculate_buffer(gdf_lu, gdf_hct.copy(), distance=5280.0/2)

In [23]:
pd.options.display.float_format = '{:0,.0f}'.format
col_dict = {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
            'equity_focus_areas_2023__efa_dis': 'Disability',
           'rg_proposed': 'Regional Geography',
            'CountyName': 'County',
            'GrowthCenterName': 'Regional Growth Center',
            'RGC_binary': 'RGC Binary',
            'Region': 'Region'
           }

hct_hh_df = pd.DataFrame()
df_025 = df_025.copy()
df_025['Region'] = 1
df = aggregate_parcels(df_025, col_dict, 'hh_p')
df['distance'] = 0.25
hct_hh_df = pd.concat([hct_hh_df,df])
df_050 = df_050.copy()
df_050['Region'] = 1
df = aggregate_parcels(df_050, col_dict, 'hh_p')
df['distance'] = 0.50
hct_hh_df = pd.concat([hct_hh_df,df])

In [24]:
pd.options.display.float_format = '{:0,.0f}'.format

In [25]:

df = hct_hh_df[hct_hh_df['group'] == 'Region']
df = df.rename(columns={'hh_p': 'Total'}).copy()
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Region'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df = df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'})

df['% of Households (1/4 Mile)'] = df['1/4 Mile']/df_lu['hh_p'].sum()
df['% of Households (1/2 Mile)'] = df['1/2 Mile']/df_lu['hh_p'].sum()

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Region,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
1,"663,997","1,022,978",27.4%,42.3%


In [26]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'County'].copy()
df = df.rename(columns={'hh_p': 'Total'}).copy()
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'County'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

_df = df.merge(df_lu[['CountyName','hh_p']].groupby('CountyName').sum()[['hh_p']].reset_index(),left_on='County', right_on='CountyName')
_df['% of Households (1/4 Mile)'] = _df['1/4 Mile']/_df['hh_p']
_df['% of Households (1/2 Mile)'] = _df['1/2 Mile']/_df['hh_p']

df = df.merge(_df[['County', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='County')

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

County,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
King,"513,012","733,678",39.8%,56.9%
Kitsap,848,"5,566",0.6%,3.7%
Pierce,"46,373","79,884",9.3%,16.0%
Snohomish,"103,764","203,850",21.5%,42.3%


In [27]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'RGC Binary'].copy()
df.rename(columns={'hh_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'RGC Binary'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)


_df = df.merge(df_lu[['RGC_binary','hh_p']].groupby('RGC_binary').sum()[['hh_p']].reset_index(),left_on='RGC Binary', 
               right_on='RGC_binary')
_df['% of Households (1/4 Mile)'] = _df['1/4 Mile']/_df['hh_p']
_df['% of Households (1/2 Mile)'] = _df['1/2 Mile']/_df['hh_p']

df = df.merge(_df[['RGC Binary', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='RGC Binary')
df['RGC Designation'] = df['RGC Binary'].map({0: 'Outside RGC', 1: 'Inside RGC'})
df = df[['RGC Designation','1/4 Mile','1/2 Mile','% of Households (1/4 Mile)', '% of Households (1/2 Mile)']]
output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

RGC Designation,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Outside RGC,"349,150","622,998",17.7%,31.5%
Inside RGC,"314,847","399,980",70.6%,89.7%


In [28]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'Regional Growth Center'].copy()
df.rename(columns={'hh_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Regional Growth Center'
# Ensure all RGCs are included, even those that have no HCT access
for center in parcel_geog['GrowthCenterName'].unique():
    if center not in df.index:
        df.loc[center] = 0

df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

# Calcualte the percent of households in this geography within 1/4 and 1/2 mile
_df = df.merge(df_lu[['GrowthCenterName','hh_p']].groupby('GrowthCenterName').sum()[['hh_p']].reset_index(),left_on='Regional Growth Center', 
               right_on='GrowthCenterName')
_df['% of Households (1/4 Mile)'] = _df['1/4 Mile']/_df['hh_p']
_df['% of Households (1/2 Mile)'] = _df['1/2 Mile']/_df['hh_p']

df = df.merge(_df[['Regional Growth Center', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='Regional Growth Center')

df = df.sort_values('Regional Growth Center')

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Regional Growth Center,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Auburn,"4,805","5,005",96.0%,100.0%
Bellevue,"32,173","32,904",97.8%,100.0%
Bothell Canyon Park,729,"3,438",13.7%,64.7%
Bremerton,266,"2,641",4.9%,49.0%
Burien,"8,537","8,803",97.0%,100.0%
Everett,"11,135","20,877",49.7%,93.2%
Federal Way,"5,949","6,226",95.6%,100.0%
Greater Downtown Kirkland,"5,468","6,157",88.7%,99.9%
Issaquah,"1,196","2,634",45.4%,100.0%
Kent,"2,156","3,743",55.9%,97.1%


In [29]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'Regional Geography'].copy()
df.rename(columns={'hh_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Regional Geography'
df.reset_index(inplace=True)

df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
HTML(df.to_html(index=False))

_df = df.merge(df_lu[['rg_proposed','hh_p']].groupby('rg_proposed').sum()[['hh_p']].reset_index(),left_on='Regional Geography', 
               right_on='rg_proposed')
_df['% of Households (1/4 Mile)'] = _df['1/4 Mile']/_df['hh_p']
_df['% of Households (1/2 Mile)'] = _df['1/2 Mile']/_df['hh_p']

df = df.merge(_df[['Regional Geography', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='Regional Geography')

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Regional Geography,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Cities and Towns,464,"2,068",0.3%,1.2%
Core Cities,"150,164","259,914",26.6%,46.0%
High Capacity Transit Communities,"93,234","165,992",18.1%,32.2%
Metropolitan Cities,"416,872","585,499",48.8%,68.6%
Rural Areas,687,"2,301",0.3%,1.0%
Urban Unincorporated Areas,"2,576","7,204",3.1%,8.7%


In [30]:
pd.options.display.float_format = '{:0,.2f}'.format
# df = hct_hh_df.copy()
# df = df[df['index'] == 1]
# df.rename(columns={'hh_p': 'Total'}, inplace=True)
# df.drop('index', axis=1, inplace=True)
# df = df.pivot_table(index='group', columns='distance', values='Total', aggfunc='sum')
# df[0.25] = df[0.25].astype('float32')
# df[0.5] = df[0.5].astype('float32')
# df.index.name = 'Equity Geography'
# df = df.reset_index()
# df = df.rename_axis(None, axis=1)
# df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

In [31]:
pd.options.display.float_format = '{:0,.1f}'.format

equity_dict = {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
            'equity_focus_areas_2023__efa_dis': 'Disability'}

df = hct_hh_df[hct_hh_df['group'].isin(equity_dict.values())].copy()
df['EFA Type'] = df['index'].map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })

df = df.pivot_table(index=['group','EFA Type'], columns='distance', values='hh_p', aggfunc='sum').reset_index()

# Merge with total households to get shares of group
efa_pop_tot = df_lu[equity_dict.keys()].sum().to_dict()

for lu_col, label in equity_dict.items():
    df.loc[df['group'] == label, "Total Pop"] = efa_pop_tot[lu_col]
df['% of Households (1/4 Mile)'] = df[0.25]/df['Total Pop']
df['% of Households (1/2 Mile)'] = df[0.5]/df['Total Pop']

df.rename(columns={'group': 'Group', 0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
df = df.rename_axis(None, axis=1)
df.drop('Total Pop', inplace=True, axis=1)
output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Group,EFA Type,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Disability,Above Regional Average,194211,304358,24.6%,38.5%
Disability,Below Regional Average,320608,503626,40.6%,63.7%
Disability,Higher Share of Equity Population,149178,214994,18.9%,27.2%
LEP,Above Regional Average,185556,275400,32.2%,47.7%
LEP,Below Regional Average,306195,476350,53.1%,82.6%
LEP,Higher Share of Equity Population,172246,271228,29.9%,47.0%
Older,Above Regional Average,157949,247112,16.9%,26.4%
Older,Below Regional Average,418965,650739,44.8%,69.6%
Older,Higher Share of Equity Population,87083,125127,9.3%,13.4%
People of Color,Above Regional Average,252692,383219,39.8%,60.3%


## Jobs Near HCT

In [32]:
pd.options.display.float_format = '{:0,.0f}'.format
col_dict = {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
            'equity_focus_areas_2023__efa_dis': 'Disability',
           'rg_proposed': 'Regional Geography',
            'CountyName': 'County',
            'GrowthCenterName': 'Regional Growth Center',
            'RGC_binary': 'RGC Binary',
            'Region': 'Region'
           }

hct_hh_df = pd.DataFrame()
df_025['Region'] = 1
df = aggregate_parcels(df_025, col_dict, 'emptot_p')
df['distance'] = 0.25
hct_hh_df = pd.concat([hct_hh_df,df])
df_050['Region'] = 1
df = aggregate_parcels(df_050, col_dict, 'emptot_p')
df['distance'] = 0.50
hct_hh_df = pd.concat([hct_hh_df,df])

In [33]:

df = hct_hh_df[hct_hh_df['group'] == 'Region'].copy()
df.rename(columns={'emptot_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Region'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

df['% of Jobs (1/4 Mile)'] = df['1/4 Mile']/df_lu['emptot_p'].sum()
df['% of Jobs (1/2 Mile)'] = df['1/2 Mile']/df_lu['emptot_p'].sum()

output = df.to_html(formatters={
    '% of Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Region,1/4 Mile,1/2 Mile,% of Jobs (1/4 Mile),% of Jobs (1/2 Mile)
1,"1,450,599","1,992,888",45.7%,62.8%


In [34]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'County'].copy()
df.rename(columns={'emptot_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'County'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

_df = df.merge(df_lu[['CountyName','emptot_p']].groupby('CountyName').sum()[['emptot_p']].reset_index(),left_on='County', right_on='CountyName')
_df['% of Jobs (1/4 Mile)'] = _df['1/4 Mile']/_df['emptot_p']
_df['% of Jobs (1/2 Mile)'] = _df['1/2 Mile']/_df['emptot_p']

df = df.merge(_df[['County', '% of Jobs (1/4 Mile)', '% of Jobs (1/2 Mile)']], 
              on='County')

output = df.to_html(formatters={
    '% of Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

County,1/4 Mile,1/2 Mile,% of Jobs (1/4 Mile),% of Jobs (1/2 Mile)
King,"1,179,346","1,533,557",57.6%,74.9%
Kitsap,"4,145","21,505",2.8%,14.6%
Pierce,"101,950","150,428",21.0%,31.0%
Snohomish,"165,158","287,398",33.5%,58.3%


In [35]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'RGC Binary'].copy()
df.rename(columns={'emptot_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'RGC Binary'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)


_df = df.merge(df_lu[['RGC_binary','emptot_p']].groupby('RGC_binary').sum()[['emptot_p']].reset_index(),left_on='RGC Binary', 
               right_on='RGC_binary')
_df['% of Jobs (1/4 Mile)'] = _df['1/4 Mile']/_df['emptot_p']
_df['% of Jobs (1/2 Mile)'] = _df['1/2 Mile']/_df['emptot_p']

df = df.merge(_df[['RGC Binary', '% of Jobs (1/4 Mile)', '% of Jobs (1/2 Mile)']], 
              on='RGC Binary')
df['RGC Designation'] = df['RGC Binary'].map({0: 'Outside RGC', 1: 'Inside RGC'})
df = df[['RGC Designation','1/4 Mile','1/2 Mile','% of Jobs (1/4 Mile)', '% of Jobs (1/2 Mile)']]
output = df.to_html(formatters={
    '% of Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

RGC Designation,1/4 Mile,1/2 Mile,% of Jobs (1/4 Mile),% of Jobs (1/2 Mile)
Outside RGC,"524,224","891,013",26.7%,45.4%
Inside RGC,"926,375","1,101,875",76.6%,91.1%


In [36]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'Regional Growth Center'].copy()
df.rename(columns={'emptot_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Regional Growth Center'
# Ensure all RGCs are included, even those that have no HCT access
for center in parcel_geog['GrowthCenterName'].unique():
    if center not in df.index:
        df.loc[center] = 0
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
df['% of Total Jobs (1/4 Mile)'] = df['1/4 Mile']/df_lu['emptot_p'].sum()
df['% of Total Jobs (1/2 Mile)'] = df['1/2 Mile']/df_lu['emptot_p'].sum()

output = df.to_html(formatters={
    '% of Total Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Total Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Regional Growth Center,1/4 Mile,1/2 Mile,% of Total Jobs (1/4 Mile),% of Total Jobs (1/2 Mile)
Auburn,"9,039","9,693",0.3%,0.3%
Bellevue,"97,138","98,694",3.1%,3.1%
Bothell Canyon Park,"3,600","11,345",0.1%,0.4%
Bremerton,"1,349","14,614",0.0%,0.5%
Burien,"8,680","8,962",0.3%,0.3%
Everett,"36,484","51,927",1.1%,1.6%
Federal Way,"11,815","12,628",0.4%,0.4%
Greater Downtown Kirkland,"19,743","20,446",0.6%,0.6%
Issaquah,"3,217","14,131",0.1%,0.4%
Kent,"24,641","33,032",0.8%,1.0%


In [37]:
df = hct_hh_df[hct_hh_df['group'] == 'Regional Geography'].copy()
df.rename(columns={'emptot_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Regional Geography'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
df['% of Total Jobs (1/4 Mile)'] = df['1/4 Mile']/df_lu['emptot_p'].sum()
df['% of Total Jobs (1/2 Mile)'] = df['1/2 Mile']/df_lu['emptot_p'].sum()

output = df.to_html(formatters={
    '% of Total Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Total Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Regional Geography,1/4 Mile,1/2 Mile,% of Total Jobs (1/4 Mile),% of Total Jobs (1/2 Mile)
Cities and Towns,847,"2,414",0.0%,0.1%
Core Cities,"389,956","629,114",12.3%,19.8%
High Capacity Transit Communities,"89,634","140,410",2.8%,4.4%
Metropolitan Cities,"963,847","1,205,152",30.4%,38.0%
Rural Areas,"2,400","6,037",0.1%,0.2%
Urban Unincorporated Areas,"3,915","9,761",0.1%,0.3%


In [38]:
pd.options.display.float_format = '{:0,.0f}'.format

equity_dict = {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
            'equity_focus_areas_2023__efa_dis': 'Disability'}

df = hct_hh_df[hct_hh_df['group'].isin(equity_dict.values())].copy()
df['EFA Type'] = df['index'].map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })

df = df.pivot_table(index=['group','EFA Type'], columns='distance', values='emptot_p', aggfunc='sum').reset_index()

# Merge with total households to get shares of group
efa_pop_tot = df_lu[equity_dict.keys()].sum().to_dict()

df[0.25] = df[0.25].astype("float")
df[0.50] = df[0.50].astype("float")

for lu_col, label in equity_dict.items():
    df.loc[df['group'] == label, "Total Jobs"] = efa_pop_tot[lu_col]
df['% of Households (1/4 Mile)'] = df[0.25]/df['Total Jobs']
df['% of Households (1/2 Mile)'] = df[0.5]/df['Total Jobs']


df.rename(columns={'group': 'Group', 0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
df = df.rename_axis(None, axis=1)
df.drop('Total Jobs', inplace=True, axis=1)
output = df.to_html(formatters={
    '% of Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Group,EFA Type,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Disability,Above Regional Average,"383,327","532,110",0,1
Disability,Below Regional Average,"636,467","931,353",1,1
Disability,Higher Share of Equity Population,"430,805","529,425",1,1
LEP,Above Regional Average,"434,539","570,947",1,1
LEP,Below Regional Average,"650,303","890,045",1,2
LEP,Higher Share of Equity Population,"365,757","531,896",1,1
Older,Above Regional Average,"376,429","498,804",0,1
Older,Below Regional Average,"913,915","1,275,211",1,1
Older,Higher Share of Equity Population,"160,255","218,873",0,0
People of Color,Above Regional Average,"638,941","807,902",1,1


## Transit Access Mode
Percent of Transit Trips Accessed by Walking

In [39]:
pd.options.display.float_format = '{:0,.1%}'.format
df = pd.read_csv(r'../../../../outputs/agg/dash/trip_mode_by_tour_mode.csv')
_df = df.copy()
pnr_transit_trips = _df[(_df['tmodetp'] == 'Park') & (_df['mode'] == 'Transit')]['trexpfac'].sum()
walk_transit_trips = _df[(_df['tmodetp'] == 'Transit') & (_df['mode'] == 'Transit')]['trexpfac'].sum()
reg_access = (walk_transit_trips/(walk_transit_trips+pnr_transit_trips))
pd.DataFrame([reg_access], columns=['% Transit trips Accessed by Walking'], index=['Region'])


,% Transit trips Accessed by Walking
Region,96.4%


In [40]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = df.copy()
results_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']
# for col in ['hh_disability','hh_elderly','hh_english','hh_poverty','hh_racial','hh_youth']:
for col in summary_config["hh_equity_geogs"]:
    pnr_transit_trips = _df[(_df[col] == 1) & 
                            (_df['tmodetp'] == 'Park') &
                            (_df['mode'] == 'Transit')]['trexpfac'].sum()
    walk_transit_trips = _df[(_df[col] == 1) & 
                                (_df['tmodetp'] == 'Transit') & 
                                (_df['mode'] == 'Transit')]['trexpfac'].sum()
    if (pnr_transit_trips > 0) & (walk_transit_trips > 0):
        results_df.loc[col,'% Transit Trips Accessed by Walking'] = walk_transit_trips/(walk_transit_trips+pnr_transit_trips)
# results_df.rename(columns={'_reg':'> Regional Average', '_50': '> 50%'}, inplace=True)
results_df

,% Transit Trips Accessed by Walking
hh_efa_dis,96.5%
hh_efa_older,95.8%
hh_efa_lep,96.5%
hh_efa_pov200,96.6%
hh_efa_poc,96.5%
hh_efa_youth,94.8%


## Transit Equity

### Trip purpose by transit mode type

In [41]:


myh5 = h5py.File(r'..\..\..\..\inputs\scenario\landuse\hh_and_persons.h5', 'r')
person = pd.read_csv(r'..\..\..\..\outputs\daysim\_person.tsv', sep='\t')
df_trip = pd.read_csv(r'..\..\..\..\outputs\daysim\_trip.tsv', sep='\t')
df_hh = pd.read_csv(r'..\..\..\..\outputs\daysim\_household.tsv', sep='\t')

In [42]:
# Create a DataFrame from the h5 file
df_person = pd.DataFrame()
for col in ['hhno','pno','prace']:
    df_person[col] = myh5['Person'][col][:]

df_trip = df_trip.merge(df_person[['hhno','pno','prace']], on=['hhno','pno'], how='left')

In [43]:
# Create group of work, school, and all other purposes
df_trip['Purpose Type'] = 'Other'
df_trip.loc[df_trip['dpurp']==0, 'Purpose Type'] = 'Home'
df_trip.loc[df_trip['dpurp']==1, 'Purpose Type'] = 'Work'
df_trip.loc[df_trip['dpurp']==2, 'Purpose Type'] = 'School'

# Rename path types
df_trip.loc[df_trip['pathtype']==1, 'Path Type'] = 'Drive'
df_trip.loc[df_trip['pathtype']==3, 'Path Type'] = 'Bus'
df_trip.loc[df_trip['pathtype']==4, 'Path Type'] = 'Light Rail'
df_trip.loc[df_trip['pathtype']==5, 'Path Type'] = 'Ferry 1'
df_trip.loc[df_trip['pathtype']==6, 'Path Type'] = 'Commuter Rail'
df_trip.loc[df_trip['pathtype']==7, 'Path Type'] = 'Ferry 2'

df = pd.pivot_table(df_trip[(df_trip['dpurp']!=0) & (~df_trip['pathtype'].isin([0,'Drive']))], values='trexpfac', index='Path Type', columns='Purpose Type', aggfunc='sum')

df['Total'] = df.sum(axis=1)
for row in df.index:
    for col in ['Work','School','Other']:
        df.loc[row, col + ' %'] = df.loc[row, col] / df.loc[row, 'Total']

df[['Work %','School %','Other %']] = df[['Work %','School %','Other %']].map("{:,.1%}".format)
df[['Work %','School %','Other %']]

Purpose Type,Work %,School %,Other %
Path Type,,,
Bus,14.3%,12.9%,72.8%
Commuter Rail,51.5%,2.5%,46.1%
Drive,20.6%,5.5%,73.9%
Ferry 1,66.7%,5.8%,27.5%
Ferry 2,51.8%,8.9%,39.3%
Light Rail,30.7%,12.7%,56.5%


In [44]:

race_dict = {
    '1': 'White alone',
    '2': 'Black or African American alone',
    '3': 'American Indian alone',
    '4': 'Alaska Native alone',
    '5': 'American Indian and Alaska Native tribes specified',
    '6': 'Asian alone',
    '7': 'Native Hawaiian and Other Pacific Islander alone',
    '8': 'Some Other Race alone',
    '9': 'Two or More Races'
}


df_trip['Race'] = df_trip['prace'].astype('int').astype('str').map(race_dict)

### Transit type distribution by race of rider

In [45]:
# Submode Ridership by race
df = pd.pivot_table(df_trip[(df_trip['pathtype']!="Drive")], values='trexpfac', index='Path Type', columns='Race', aggfunc="sum")
race_cols = ['White alone','Black or African American alone','American Indian alone',
             'Alaska Native alone','American Indian and Alaska Native tribes specified','Asian alone',
             'Native Hawaiian and Other Pacific Islander alone','Some Other Race alone','Two or More Races']
# Calculate shares of total by Race for each Path Type
df['Total'] = df.sum(axis=1)
df = df.astype('float')
for row in df.index:
    for col in race_cols:
        df.loc[row, col] = df.loc[row, col] / df.loc[row, 'Total']
df[race_cols] = df[race_cols].map("{:,.1%}".format)
df[race_cols]

Race,White alone,Black or African American alone,American Indian alone,Alaska Native alone,American Indian and Alaska Native tribes specified,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,Two or More Races
Path Type,,,,,,,,,
Bus,63.3%,6.7%,0.6%,0.1%,0.1%,15.7%,0.7%,4.0%,8.8%
Commuter Rail,64.9%,7.1%,0.9%,0.1%,0.1%,12.2%,1.3%,5.3%,8.1%
Drive,65.2%,5.7%,0.7%,0.1%,0.1%,14.3%,0.9%,4.3%,8.8%
Ferry 1,70.0%,4.8%,0.7%,0.0%,0.1%,13.1%,0.5%,3.1%,7.7%
Ferry 2,73.1%,3.8%,0.7%,0.1%,0.1%,9.8%,0.7%,3.5%,8.1%
Light Rail,62.5%,6.4%,0.6%,0.1%,0.1%,17.1%,0.7%,3.9%,8.6%


In [46]:
df_trip = df_trip.merge(df_hh[['hhno','hhincome']], on=['hhno'], how='left')

### Income distribution by household income of rider

In [47]:
# Create bins for hhincome
bins = [0, 20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000, 180000, 200000, np.inf]
labels = ['<20k', '20-40k', '40-60k', '60-80k', '80-100k', '100-120k', '120-140k', '140-160k', '160-180k', '180-200k', '>200k']
df_trip['Income Bin'] = pd.cut(df_trip['hhincome'], bins=bins, labels=labels, right=False)
df_trip['Income Bin'] = df_trip['Income Bin'].astype('str')

# Transit Submodes by Income
df = pd.pivot_table(df_trip[(df_trip['Path Type']!="Drive")], values='trexpfac', index='Path Type', columns='Income Bin', aggfunc="sum")

# display labels with no significant figs
df[labels] = df[labels].map("{:0,.0f}".format)
df[labels]

Income Bin,<20k,20-40k,40-60k,60-80k,80-100k,100-120k,120-140k,140-160k,160-180k,180-200k,>200k
Path Type,,,,,,,,,,,
Bus,"60,513","58,011","53,862","48,432","47,113","38,162","35,536","30,928","25,324","23,983","123,203"
Commuter Rail,263,503,932,"1,059","1,287","1,087","1,013",881,666,689,"2,535"
Ferry 1,178,261,364,440,473,375,345,379,345,283,"1,879"
Ferry 2,"1,399","2,264","3,200","3,990","4,033","3,587","3,458","3,026","2,435","2,400","11,416"
Light Rail,"33,259","35,837","37,479","37,436","38,345","32,098","30,816","27,080","22,827","22,099","116,085"


In [48]:
pd.pivot_table(
    df_trip[df_trip['Path Type'] != "Drive"],
    values='hhincome',
    index='Path Type',
    aggfunc="median"
).astype(int).map(lambda x: "${:,.0f}".format(x))

,hhincome
Path Type,
Bus,"$102,468"
Commuter Rail,"$125,134"
Ferry 1,"$152,093"
Ferry 2,"$131,856"
Light Rail,"$121,537"
